In [1]:
import csv
from datetime import datetime, timedelta

In [ ]:

# Function to convert date format
#def convert_date(date):
#    # Convert the date string to a datetime object
#    date_object = datetime.strptime(date, '%Y/%m/%d %H:%M:%S')
#    # Format the date according to the new format
#    new_date = date_object.strftime('%Y-%m-%d_%H.%M.%S')
#    return new_date

## CSV file name
#csv_file = 'pnsn_event_export_5dias.csv'

## Lists to store converted dates and corresponding values from the first column
#converted_dates = []
#first_column_values = []

## Open the CSV file and read dates
#with open(csv_file, newline='') as csvfile:
#    csv_reader = csv.reader(csvfile)
#    # Ignore the first row (headers)
#    next(csv_reader)
#    # Iterate over the rows
#    for row in csv_reader:
#        # Get the date from the fifth column and convert it
#        original_date = row[4]
#        new_date = convert_date(original_date)
#        # Append the converted date to the list
#        converted_dates.append(new_date)
#        # Append the corresponding value from the first column
#        first_column_values.append(row[0])

## Write the converted dates and corresponding values to a text file
#with open('converted_dates_and_values_rainier_5days.txt', 'w') as txtfile:
#    for date, value in zip(converted_dates, first_column_values):
#        txtfile.write(f"{value}\t{date}\n")

#print("Process completed. 'converted_dates_and_values_rainier_5days.txt' file has been created with the converted dates and corresponding values.")


In [7]:
from datetime import datetime, timedelta

# Define the function to parse a date string into a datetime object
def parse_date(date_str):
    try:
        return datetime.strptime(date_str, '%Y-%m-%d_%H.%M.%S')
    except ValueError:
        return None

# Define the time difference allowed (5 seconds)
time_difference = timedelta(seconds=5)

# Function to read dates and values from a file and return a list of tuples containing both
def read_dates_and_values_from_file(file_path):
    dates_and_values = []
    with open(file_path, 'r') as f:
        next(f)  # Skip header line
        for line in f:
            elements = line.strip().split('\t')
            if len(elements) >= 2:  # Ensure there are at least two elements
                value = elements[0]  # Extract the value from the first column
                date_str = elements[-1]  # Extract the date from the last column
                date = parse_date(date_str)
                if date is not None:
                    dates_and_values.append((value, date))
    return dates_and_values

# Read dates and values from the converted_dates_and_values_rainier_5days.txt file
converted_dates_and_values = read_dates_and_values_from_file('converted_dates_and_values_rainier_5days.txt')

# Initialize lists to store common dates and unmatched dates
common_dates = []
unmatched_dates = []

# Read dates from the results file and compare with converted dates
with open('txtfiles/results_2023-08-27_11.38.00.txt', 'r') as f:
    next(f)  # Skip header line
    for line in f:
        columns = line.strip().split('\t')
        if len(columns) >= 3:  # Ensure there are at least three elements in the list
            threshold_value = columns[0]  # Extract threshold value from the first column
            detection_times = columns[-1].split(',')  # Extract detection times from the last column
            for detection_time in detection_times:
                result_date = parse_date(detection_time)  # Convert detection time to a datetime object
                if result_date is not None:  # Check if result_date is not None
                    # Check if the result_date has a match in converted_dates_and_values
                    matched = False
                    for value, converted_date in converted_dates_and_values:
                        if abs(result_date - converted_date) <= time_difference:
                            common_dates.append((threshold_value, result_date, value))
                            matched = True
                            break
                    if not matched:
                        unmatched_dates.append((threshold_value, result_date))

# Write common and unmatched dates to a single file
with open('output.txt', 'w') as f:
    f.write("Threshold\tResult Date\tID Value\tMatch Status\n")
    for threshold_value, result_date, id_value in common_dates:
        f.write(f"{threshold_value}\t{result_date.strftime('%Y-%m-%d_%H.%M.%S')}\t{id_value}\tUGSGcatalog\n")
    for threshold_value, result_date in unmatched_dates:
        f.write(f"{threshold_value}\t{result_date.strftime('%Y-%m-%d_%H.%M.%S')}\tX\tUnmatched\n")

print("Output file 'output.txt' has been generated.")


Output file 'output.txt' has been generated.
